In [3]:
from aocd import submit
import numpy as np
import re
from itertools import product

In [4]:
YYYY=2021
DD=22

In [5]:
def translate(s):
    parse=re.compile("[-+]?[0-9]+")
    s1 = 1 if  s[0:2] == "on" else 0
    s2 = [int(n) for n in parse.findall(s)]
    s2 = [(s2[0], s2[1]), (s2[2], s2[3]), (s2[4], s2[5])]

    return [s1, tuple(s2)]
    
def load(filename):
    with open(filename,'r') as f:
         lines = [translate(s)  for s in f]
    return lines
test1 = load("test1.dat")
test2 = load("test2.dat")
test3 = load("test3.dat")
test2

[[1, ((-20, 26), (-36, 17), (-47, 7))],
 [1, ((-20, 33), (-21, 23), (-26, 28))],
 [1, ((-22, 28), (-29, 23), (-38, 16))],
 [1, ((-46, 7), (-6, 46), (-50, -1))],
 [1, ((-49, 1), (-3, 46), (-24, 28))],
 [1, ((2, 47), (-22, 22), (-23, 27))],
 [1, ((-27, 23), (-28, 26), (-21, 29))],
 [1, ((-39, 5), (-6, 47), (-3, 44))],
 [1, ((-30, 21), (-8, 43), (-13, 34))],
 [1, ((-22, 26), (-27, 20), (-29, 19))],
 [0, ((-48, -32), (26, 41), (-47, -37))],
 [1, ((-12, 35), (6, 50), (-50, -2))],
 [0, ((-48, -32), (-32, -16), (-15, -5))],
 [1, ((-18, 26), (-33, 15), (-7, 46))],
 [0, ((-40, -22), (-38, -28), (23, 41))],
 [1, ((-16, 35), (-41, 10), (-47, 6))],
 [0, ((-32, -23), (11, 30), (-14, 3))],
 [1, ((-49, -5), (-3, 45), (-29, 18))],
 [0, ((18, 30), (-20, -8), (-3, 13))],
 [1, ((-41, 9), (-7, 43), (-33, 15))],
 [1, ((-54112, -39298), (-85059, -49293), (-27449, 7877))],
 [1, ((967, 23432), (45373, 81175), (27513, 53682))]]

In [6]:
data_in = load("input.dat")

# Teil a

In [5]:
"""
0..100
-50 = 0
50 = 100
"""
a= 'on x=10..12,y=10..12,z=10..12'
parse=re.compile("\d+")
parse.findall(a)

['10', '12', '10', '12', '10', '12']

In [6]:
def transform(cubes):
    c=list(cubes)
    for i,v in enumerate(cubes):
        c[i] = 50+(min(50, v) if i % 2 else max(-50,v))
    return c

In [7]:
transform(test1[0][1])

TypeError: '>' not supported between instances of 'tuple' and 'int'

In [8]:
def solution_a(data):
    c = np.zeros((101,101,101), int)
    for s, d in data:
        x1,x2, y1,y2, z1, z2 = transform(d)
        c[x1:x2+1,y1:y2+1,z1:z2+1] = s
        #print(c.sum())
    return c.sum()

In [9]:
solution_a(test2)

TypeError: '>' not supported between instances of 'tuple' and 'int'

In [10]:
submit(solution_a(data_in),part="a",day=DD, year=YYYY)

TypeError: '>' not supported between instances of 'tuple' and 'int'

# Teil b

In [8]:
def isin(a, b):
    # is a in b?
    if a == "v":
        return False
    a1,a2=a
    b1,b2=b
    return (b1 <= a1 and a2 <= b2)    


def create_interval(l,r):
    if r-l > 1:
        return set([(l,l), (l+1,r-1), (r,r)])
    return set([(l,l), (r,r)])
    
def list2interval(l):
    result = set()
    for l,r in zip(l[:-1],l[1:]):
        result |= create_interval(l,r)
    return list(result)
    
            
def init(data):
    X,Y,Z = set(),set(),set()
    for switch, cube in data:
        x,y,z = cube
        X|=set([x[0],x[1]])
        Y|=set([y[0],y[1]])
        Z|=set([z[0],z[1]])
    X=list2interval(sorted(list(X)))
    Y=list2interval(sorted(list(Y)))
    Z=list2interval(sorted(list(Z)))
    return sorted(X),sorted(Y),sorted(Z)

def get_volumns(x,y,z):
    x=np.array([1+x1-x0 for x0,x1 in x])
    y=np.array([1+y1-y0 for y0,y1 in y])
    z=np.array([1+z1-z0 for z0,z1 in z])
    X,Y,Z = np.ix_(x,y,z)
    return X*Y*Z

def split_cube2(cube, X,Y,Z):
    x,y,z = cube
    _x=[xx for i, xx in  enumerate(X) if isin(xx, x)]
    _y=[yy for i, yy in  enumerate(Y) if isin(yy, y)]
    _z=[zz for i, zz in  enumerate(Z) if isin(zz, z)]
    return _x, _y, _z

def get_cube(cube, X,Y,Z):
    x,y,z = cube
    _x=[i for i, xx in  enumerate(X) if isin(xx, x)]
    _y=[i for i, yy in  enumerate(Y) if isin(yy, y)]
    _z=[i for i, zz in  enumerate(Z) if isin(zz, z)]
    return ((min(_x), max(_x)),(min(_y), max(_y)),(min(_z), max(_z)))

    
def count2(meta, x,y,z):
    s = 0
    dimx,dimy,dimz=meta.shape
    
    for xx in range(dimx):
        print(xx)
        xxx = (1+x[xx][1]-x[xx][0])
        for yy in range(dimy):
            yyy = (1+y[yy][1]-y[yy][0])
            for zz in range(dimz):
                if meta[xx,yy,zz]:
                    s += xxx*yyy*(1+z[zz][1]-z[zz][0])
    return s

def count(meta, vols):    
    for i in range(meta.shape[0]):
        vols[i,:,:] *= meta[i,:,:]

                

def solution_b(data):
    x,y,z = init(data)
    dimx, dimy, dimz = len(x), len(y), len(z)
    meta = np.zeros((dimx, dimy, dimz), bool)
    volumns = get_volumns(x,y,z)
    for i, (switch, cube) in enumerate(data):
        # print(i,cube)
        (x1,x2),(y1,y2),(z1,z2) = get_cube(cube, x,y,z)
        meta[x1:x2+1,y1:y2+1,z1:z2+1] = switch 
    print("calc volumn ... may take a while")
    count(meta, volumns)
    return volumns.sum()
                    

In [12]:
%%time
solution_b(test3)

calc volumn ... may take a while
CPU times: user 50.2 ms, sys: 31.9 ms, total: 82.1 ms
Wall time: 80.3 ms


2758514936282235

In [13]:
%%time
solution_b(data_in)

calc volumn ... may take a while
CPU times: user 1min 27s, sys: 4min 11s, total: 5min 39s
Wall time: 25min 26s


1263946820845866

In [50]:
i+i

array([[[ 48,  56,  64,  72],
        [ 60,  70,  80,  90]],

       [[ 96, 112, 128, 144],
        [120, 140, 160, 180]],

       [[144, 168, 192, 216],
        [180, 210, 240, 270]]])